In [1]:
# Run in terminal or command prompt
!pip install spacy
!python3 -m spacy download en

  Using cached https://files.pythonhosted.org/packages/35/d5/4f8410ac303e690144f0a0603c4b8fd3b986feb2749c435f7cdbb288f17e/numpy-1.16.2-cp36-cp36m-manylinux1_x86_64.whl
featuretools 0.4.1 has requirement pandas>=0.23.0, but you'll have pandas 0.22.0 which is incompatible.
datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.8 which is incompatible.
  Found existing installation: numpy 1.14.6
    Uninstalling numpy-1.14.6:
      Successfully uninstalled numpy-1.14.6



    Linking successful
    /usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/en

    You can now load the model via spacy.load('en')



In [2]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
#import pyLDAvis
#import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline


paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress


# Prepare NLTK stop words

In [3]:
import nltk
nltk.download('stopwords')
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'The New York Times'])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
!head -n 1 news.csv

e compounds are, in turn, the basic energy supply for almost all animal cells, including those of humans. The mathematical description of photosynthesis is sometimes billed as “the equation that powers the world. ” For a decade, Dr. Long had argued that photosynthesis was not actually very efficient. In the course of evolution, several experts said, Mother Nature had focused on the survival and reproduction of plants, not on putting out the maximum amount of seeds or fruits for humans to come along and pick. Dr. Long thought crop yields might be improved by certain genetic changes. Other scientists doubted it would work, but with the Science paper, Dr. Long and his collaborator  —   Krishna K. Niyogi, who holds appointments at the University of California, Berkeley, and the Lawrence Berkeley National Laboratory  —   have gone a long way toward proving their point. Much of the work at the University of Illinois was carried out by two young researchers from abroad who hold positions in D

In [9]:
df = pd.read_csv('news.csv', header=0)
# Convert to list
data = df.content.values.tolist()

AttributeError: ignored

In [8]:
df.head()

,e compounds are,in turn,the basic energy supply for almost all animal cells,including those of humans. The mathematical description of photosynthesis is sometimes billed as “the equation that powers the world. ” For a decade,Dr. Long had argued that photosynthesis was not actually very efficient. In the course of evolution,several experts said,Mother Nature had focused on the survival and reproduction of plants,not on putting out the maximum amount of seeds or fruits for humans to come along and pick. Dr. Long thought crop yields might be improved by certain genetic changes. Other scientists doubted it would work,but with the Science paper,Dr. Long and his collaborator — Krishna K. Niyogi,...,assuming its early promise is borne out,make its way to African farmers at low cost. The work is,in part,an effort to secure the food supply against the possible effects of future climate change. If rising global temperatures cut the production of food,human society could be destabilized,but more efficient crop plants could potentially make the food system more resilient,Dr. Long said. “We’re in a year when commodity prices are very low,and people are saying the world doesn’t need more food,” Dr. Long said. “But if we don’t do this now,"we may not have it when we really need it. ”"""
0,4605,22384,Where to Be Single in New York - The New York ...,New York Times,Kim Velsey,2017-03-31,2017.0,3.0,NaN,There may be no such thing as an ideal neighbo...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4606,22385,When Eve and Eve Bit the Apple - The New York ...,New York Times,Kristen Scharold,2017-04-06,2017.0,4.0,NaN,When you are raised to be a good Christian gir...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4607,22386,Trump Turns to His Right Flank to Fill Nationa...,New York Times,Julie Hirschfeld Davis,2017-04-29,2017.0,4.0,NaN,WASHINGTON — Donald J. Trump moved quickl...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4608,22387,Ivanka Trump’s Presence at Meeting With Japan’...,New York Times,Eric Lipton,2017-04-19,2017.0,4.0,NaN,WASHINGTON — The potential for conflicts of...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4609,22388,How Could You? 19 Questions to Ask Loved Ones ...,New York Times,Michael Barbaro,2016-11-20,2016.0,11.0,NaN,Here’s what we learned after the ugliest presi...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
df.title[0]

In [0]:
df.head()

In [0]:
# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

# Remove after observing a lot of new york times
data = [re.sub("New York Times", "", sent) for sent in data]

# Tokenize words and Clean-up text

In [0]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

# Creating Bigram Model

In [0]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewe
bigram_mod = gensim.models.phrases.Phraser(bigram)

# Remove Stopwords, Make Bigrams and Lemmatize

In [0]:
# Define functions for stopwords, bigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [0]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

# Create the Dictionary and Corpus for Topic Modeling

In [0]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

In [0]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

Gensim creates a unique id for each word in the document. The produced corpus shown above is a mapping of (word_id, word_frequency).

# Building the Topic Model

In [0]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

# View the topics in LDA model

The above LDA model is built with 20 different topics where each topic is a combination of keywords and each keyword contributes a certain weightage to the topic.

In [0]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

In [0]:
!wget http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip && unzip mallet-2.0.8.zip

In [0]:
!ls

#  Building LDA Mallet Model

Gensim provides a wrapper to implement Mallet’s LDA from within Gensim itself. 

In [0]:
mallet_path = 'mallet-2.0.8/bin/mallet'

In [0]:
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)


In [0]:
pprint(ldamallet.show_topics(formatted=False))

# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

 # Find the optimal number of topics for LDA
 
 My approach to finding the optimal number of topics is to build many LDA models with different values of number of topics (k) and pick the one that gives the highest coherence value.

Choosing a ‘k’ that marks the end of a rapid growth of topic coherence usually offers meaningful and interpretable topics. Picking an even higher value can sometimes provide more granular sub-topics.

If  the same keywords is repeated in multiple topics, it’s probably a sign that the ‘k’ is too large.

The compute_coherence_values()  trains multiple LDA models and provides the models and their corresponding coherence scores.

In [0]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):

    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [0]:
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=25, limit=60, step=10)

In [0]:
# Show graph
limit=60; start=25; step=10;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

In [0]:
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

In [0]:
# Select the model and print the topics
optimal_model = model_list[4]
model_topics = optimal_model.show_topics(formatted=False)
pprint(optimal_model.print_topics(num_words=10))

# Explain what can you find from the results

Some of the topic can be easily found

No.24: US election

No.12: politics

No.1: Military

We can also see some of topics are overlaped.

# Finding the dominant topic in each sentence

To find that, we find the topic number that has the highest percentage contribution in that document.

In [0]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=data)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)

We can see that for each sentence, it actually has keywords that conrespond to relevant topics. 

# Find the most representative document for each topic

Find the documents a given topic has contributed to the most and infer the topic by reading that document.

In [0]:
# Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]

# Show
sent_topics_sorteddf_mallet

Each row above corresponds to one topic. It has the topic number, the keywords, and the most representative document. The Perc_Contribution column is nothing but the percentage contribution of the topic in the given document.

# Topic distribution across documents

Finally, we want to understand the volume and distribution of topics in order to judge how widely it was discussed. The below table exposes that information.

In [0]:
# Number of Documents for Each Topic
topic_counts = df_topic_sents_keywords['Dominant_Topic'].value_counts()

# Percentage of Documents for Each Topic
topic_contribution = round(topic_counts/topic_counts.sum(), 4)

# Topic Number and Keywords
topic_num_keywords = df_topic_sents_keywords[['Dominant_Topic', 'Topic_Keywords']]

# Concatenate Column wise
df_dominant_topics = pd.concat([topic_num_keywords, topic_counts, topic_contribution], axis=1)

# Change Column names
df_dominant_topics.columns = ['Dominant_Topic', 'Topic_Keywords', 'Num_Documents', 'Perc_Documents']

# Show
df_dominant_topics[:29]


We can see that the percentage of documents for each topic at the last column. 

Most of them cover the range between 2% to 4%